In [1]:
from script.save_interpret import save_to_csv
save_to_csv("classification", "quality")

  0%|          | 0/252 [00:00<?, ?it/s]

NameError: name 'explaination' is not defined

In [2]:
a = [(1,2),(3,4),(5,6)]
for x,y in a:
    print(x)
    print(y)

1
2
3
4
5
6


In [79]:
from tqdm import tqdm
pred_model = model[-1].predict
pred_model

<bound method LGBMModel.predict of LGBMRegressor()>

In [80]:
 model[-1]

LGBMRegressor()

In [322]:
import numpy as np
import warnings
from lime import submodular_pick
from tqdm import tqdm
from lime.lime_tabular import LimeTabularExplainer
import pandas as pd
import joblib

X_train = pd.read_csv("/ml_data/X_train.csv").drop(columns = "Unnamed: 0")
y_train = pd.read_csv("/ml_data/y_train.csv").drop(columns = "Unnamed: 0")
X_test = pd.read_csv("/ml_data/X_test.csv").drop(columns = "Unnamed: 0")
y_test = pd.read_csv("/ml_data/y_test.csv").drop(columns = "Unnamed: 0")
feature_names = list(X_train.columns)
model = joblib.load("/artifact/mlruns/813623427044464195/69402f2e004a4032b669f7800a476315/artifacts/model/model.pkl")
# model = joblib.load("/artifact/mlruns/941649382202349625/e311be07056d4a4fa7413ec46830e13b/artifacts/model/model.pkl")
task = "regression"
target = "price"
pipeline_name = "non_remove_outlier median StandardScaler"

all_columns = []
cat_feature = []
for i in range(len(model["preprocessor"].transformers_)):
    if model["preprocessor"].transformers_[i][0] == "num":
        all_columns.extend(model["preprocessor"].transformers[i][2])
    else:
        for j in range(len(model['preprocessor'].transformers_[1][1].categories_)):
            all_columns.extend(list(model['preprocessor'].transformers_[1][1].categories_[j]))
            cat_feature.append(list(model['preprocessor'].transformers_[1][1].categories_[j]))
            
def load_transform(file_name):
    preprocessor = joblib.load(f'/save_pipeline/{file_name}.pkl')
    return preprocessor
    
if str(model[1]) ==  'KNeighborsClassifier()':
    pred_model = model[-1].predict_proba
else:
    pred_model = model[-1].predict

def check_type(df)-> list[str]:
    df_columns = list(df.columns)
    numeric = []
    catagories = []
    for values in df.columns:
        if df[values].dtypes == int or df[values].dtypes == float:
            numeric.append(values)
        else:
            catagories.append(values)
    return catagories

    
def check_file():
    path = Path("/model_interpret")
    return path.is_file()
    
def _explainer(task, target):
    if task == "classification":
        class_names = y_train[target].unique()
        # Fit the Explainer on the training data set using the LimeTabularExplainer 
        explainer = LimeTabularExplainer(load_transform(pipeline_name).fit_transform(X_train), feature_names = all_columns, class_names = class_names, mode = task, , categorical_features = cat_feature)
    else:
        explainer = LimeTabularExplainer(load_transform(pipeline_name).fit_transform(X_train), feature_names = all_columns, mode = "regression", class_names = [target], categorical_features = cat_feature)
        
    return explainer

def return_x(explaination, int_x):
    return explaination.domain_mapper.feature_names[int_x]

def return_x_2(explaination,int_x, i):
    return explaination.explanations[i].domain_mapper.feature_names[int_x]

def _to_csv_local(lime_obj, num):
    cols = ["importance_values_"+i for i in [lime_obj.domain_mapper.feature_names][0]]
    values = lime_obj.domain_mapper.feature_values
    arr = np.array(values).reshape((1, len(cols)))
    df_x = pd.DataFrame(arr , columns = cols)
    return pd.concat([X_test[X_test.index == num], df_x], axis = 1)
    

def _to_csv_global(lime_obj):
    all_df = []
    for i in range(0,5):
        cols = lime_obj.explanations[i].domain_mapper.feature_names
        values = lime_obj.explanations[i].domain_mapper.feature_values
        arr = np.array(values).reshape((1, len(list(lime_obj.explanations[i].local_exp.values())[0])))
        all_df.append(pd.DataFrame(arr , columns = cols))
    return pd.concat(all_df, ignore_index =True)

def save_to_csv(task, target):
    all_df_local = []
    num = -1
    for i in tqdm(model[0].fit_transform(X_test)):
        num+=1
        explainer = _explainer(task, target)
        explaination = explainer.explain_instance(i, pred_model)
        all_df_local.append(_to_csv_local(explaination, num))
        
    explainer = _explainer(task, target)
    sp_obj = submodular_pick.SubmodularPick(explainer, model[0].fit_transform(X_train), pred_model, num_features= len(all_columns), num_exps_desired = 5)
    all_df_global = _to_csv_global(sp_obj)
    pd.concat(all_df_local, ignore_index = True).fillna(0).to_csv("/model_interpret/local.csv")
    all_df_global.fillna(0).to_csv("/model_interpret/global.csv")


In [323]:
# all_df_local = []
# for i in tqdm(model[0].fit_transform(X_test)):
#     explainer = _explainer(task, target)
#     explaination = explainer.explain_instance(i, pred_model)
#     all_df_local.append(_to_csv_local(explaination))

In [324]:
model[0]

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', StandardScaler())]),
                                 ['bedroom', 'bathroom', 'sqm', 'built_year',
                                  'luxuary', 'public', 'BRT_distance',
                                  'BTS_distance', 'MRT_distance',
                                  'APL_distance', 'Air-Conditioning',
                                  'Audio System', 'BBQ pits',
                                  'Basketball court', 'Bathtub', 'Bed',
                                  'Billiards room', 'Broadband Internet',
                                  'Club', 'Clubhouse', 'Colonial Building',
                                  'Corner Unit', 'DVD Player', 'Dishwasher',
                                  'Driving range', 'Dryer', 'Fridge',
                 

In [325]:
model[0].fit_transform(X_test)[0]

array([-1.92133845, -0.63431634, -0.75415444, -0.30528677, -0.63828218,
       -0.48117823,  0.27562872,  1.57627616, -1.49369187,  0.37208382,
       -0.73139576, -0.05612135, -0.20834357, -0.10153462, -0.17770466,
        2.38856927, -0.02802759, -0.09751265, -0.21442251, -0.22425107,
       -0.02802759, -0.17052613, -0.02802759, -0.05612135, -0.08894601,
       -0.02802759,  2.61996136, -0.20420758, -0.18004141,  0.        ,
       -0.02802759, -0.31949354,  0.        , -0.07949286, -0.04858339,
       -0.31799936, -0.11277677, -0.09751265, -0.19135081, -0.39639612,
       -0.27072868, -0.0627703 , -0.08434823, -0.12304244, -0.2870765 ,
       -0.14990634, -0.09332428, -0.04858339,  0.        , -0.02802759,
       -0.21839727, -0.02802759, -0.26737361, -0.32246881,  0.        ,
        0.        , -0.06878853, -0.37354368, -0.05612135, -0.18004141,
        0.        , -0.28867513, -0.06878853, -0.12304244, -0.08894601,
       -0.12628921, -0.33420479, -0.44426166, -0.65074088, -0.75

In [327]:
explainer = _explainer(task, target)
explaination = explainer.explain_instance(model[0].fit_transform(X_test)[0], pred_model)
# explaination.__dict__["domain_mapper"].__dict___to_csv_local(explaination)
_to_csv_local(explaination)

eiei


TypeError: _to_csv_local() missing 1 required positional argument: 'num'

In [328]:
all_df = []
lime_obj = explaination
cols = ["importance_values_"+i for i in [lime_obj.domain_mapper.feature_names][0]]
values = lime_obj.domain_mapper.feature_values
arr = np.array(values).reshape((1, len(cols)))
df_x = pd.DataFrame(arr , columns = cols)
pd.concat([X_test[X_test.index == num], df_x], axis = 1)

NameError: name 'num' is not defined

In [329]:
pd.concat([X_test[X_test.index == 0], df_x], axis = 1)

,bedroom,bathroom,sqm,developer,built_year,district,luxuary,public,BRT_distance,BTS_distance,...,importance_values_Bang Rak,importance_values_Chatuchak,importance_values_Huai Khwang,importance_values_Khlong San,importance_values_Khlong Toei,importance_values_Pathum Wan,importance_values_Phra Khanong,importance_values_Ratchathewi,importance_values_Sathon,importance_values_Watthana
0,0,1,30.0,no,2013,Huai Khwang,0,0,10000,10000,...,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [330]:
explainer = _explainer(task, target)
sp_obj = submodular_pick.SubmodularPick(explainer, model[0].fit_transform(X_train), pred_model, num_features= len(all_columns), num_exps_desired = 5, sample_size=2)

eiei


/usr/local/lib/python3.9/site-packages/lime/submodular_pick.py:82: UserWarning: Requested number of explanations larger than
                           total number of explanations, returning all
                           explanations instead.
  warnings.warn("""Requested number of explanations larger than


In [331]:
lime_obj = sp_obj
all_df = []
for i in range(0,2):
    cols = lime_obj.explanations[i].domain_mapper.feature_names
    values = lime_obj.explanations[i].domain_mapper.feature_values
    arr = np.array(values).reshape((1, len(list(lime_obj.explanations[i].local_exp.values())[0])))
    all_df.append(pd.DataFrame(arr , columns = cols))

In [332]:
lime_obj.explanations[i].domain_mapper.feature_names

['bedroom',
 'bathroom',
 'sqm',
 'built_year',
 'luxuary',
 'public',
 'BRT_distance',
 'BTS_distance',
 'MRT_distance',
 'APL_distance',
 'Air-Conditioning',
 'Audio System',
 'BBQ pits',
 'Basketball court',
 'Bathtub',
 'Bed',
 'Billiards room',
 'Broadband Internet',
 'Club',
 'Clubhouse',
 'Colonial Building',
 'Corner Unit',
 'DVD Player',
 'Dishwasher',
 'Driving range',
 'Dryer',
 'Fridge',
 'Function room',
 'Game room',
 'Ground Floor',
 'Hairdryer',
 'High Floor',
 'ISDN',
 'Internet Connection',
 'Iron / Ironing Board',
 'Jacuzzi',
 'Karaoke',
 'Kitchen',
 'Launderette',
 'Library',
 'Lounge',
 'Low Floor',
 'Maidsroom',
 'Mini golf range',
 'Mini-Mart',
 'Multi-purpose hall',
 'Original Condition',
 'Outdoor Patio',
 'Own Chiller Plant',
 'Pantry',
 'Pavillion',
 'Penthouse',
 'Playground',
 'Private Garden',
 'Private Pool',
 'Putting Green',
 'Reflexology Path',
 'Renovated',
 'Roof Terrace',
 'SCV/SMATV System',
 'Sea View',
 'Sky Lounge',
 'Squash court',
 'Tennis cou

In [333]:
# pd.concat(all_df_local, ignore_index = True).fillna(0).to_csv("/model_interpret/local.csv")

In [334]:
save_to_csv(task, target)

  0%|          | 0/1274 [00:00<?, ?it/s]

eiei


  0%|          | 1/1274 [00:00<08:52,  2.39it/s]

eiei


  0%|          | 3/1274 [00:01<09:32,  2.22it/s]

...

eiei


  1%|          | 15/1274 [00:07<10:04,  2.08it/s]

eiei


  1%|          | 15/1274 [00:07<10:08,  2.07it/s]


KeyboardInterrupt: 

In [239]:
model['preprocessor'].transformers_[1][1].__dict__

{'categories': 'auto',
 'sparse': True,
 'dtype': numpy.float64,
 'handle_unknown': 'ignore',
 'drop': None,
 'n_features_in_': 2,
 'feature_names_in_': array(['developer', 'district'], dtype=object),
 'categories_': [array(['AP Thailand - เอพี (ไทยแลนด์) จำกัด (มหาชน)',
         'Ananda Development - อนันดา ดีเวลลอปเม้นท์ จํากัด (มหาชน)',
         'L.P.N. Development - แอล.พี.เอ็น.ดีเวลลอปเม้นท์ จำกัด (มหาชน)',
         'Land and Houses - แลนด์ แอนด์ เฮ้าส์ จำกัด (มหาชน)',
         'Magnolia Quality Development - แมกโนเลีย ควอลิตี้ ดีเวล็อปเม้นต์ คอร์ปอเรชั่น จำกัด',
         'Major Development - เมเจอร์ ดีเวลลอปเม้นท์ จำกัด (มหาชน)',
         'Noble Development - โนเบิล ดีเวลลอปเมนท์ จำกัด (มหาชน)',
         'Pace Development - เพซ ดีเวลลอปเมนท์ คอร์ปอเรชั่น จำกัด (มหาชน)',
         'Pruksa Real Estate - พฤกษา เรียลเอสเตท จำกัด (มหาชน)',
         'Quality Houses - ควอลิตี้ เฮ้าส์ จำกัด (มหาชน)',
         'Raimon Land - ไรมอน แลนด์ จำกัด (มหาชน)',
         'SC Asset - เอสซี แอสเสท คอร

In [137]:
model["preprocessor"].transformers[0][2]

['bedroom',
 'bathroom',
 'sqm',
 'built_year',
 'luxuary',
 'public',
 'BRT_distance',
 'BTS_distance',
 'MRT_distance',
 'APL_distance',
 'Air-Conditioning',
 'Audio System',
 'BBQ pits',
 'Basketball court',
 'Bathtub',
 'Bed',
 'Billiards room',
 'Broadband Internet',
 'Club',
 'Clubhouse',
 'Colonial Building',
 'Corner Unit',
 'DVD Player',
 'Dishwasher',
 'Driving range',
 'Dryer',
 'Fridge',
 'Function room',
 'Game room',
 'Ground Floor',
 'Hairdryer',
 'High Floor',
 'ISDN',
 'Internet Connection',
 'Iron / Ironing Board',
 'Jacuzzi',
 'Karaoke',
 'Kitchen',
 'Launderette',
 'Library',
 'Lounge',
 'Low Floor',
 'Maidsroom',
 'Mini golf range',
 'Mini-Mart',
 'Multi-purpose hall',
 'Original Condition',
 'Outdoor Patio',
 'Own Chiller Plant',
 'Pantry',
 'Pavillion',
 'Penthouse',
 'Playground',
 'Private Garden',
 'Private Pool',
 'Putting Green',
 'Reflexology Path',
 'Renovated',
 'Roof Terrace',
 'SCV/SMATV System',
 'Sea View',
 'Sky Lounge',
 'Squash court',
 'Tennis cou

In [150]:
model['preprocessor'].transformers_[0][2]

['bedroom',
 'bathroom',
 'sqm',
 'built_year',
 'luxuary',
 'public',
 'BRT_distance',
 'BTS_distance',
 'MRT_distance',
 'APL_distance',
 'Air-Conditioning',
 'Audio System',
 'BBQ pits',
 'Basketball court',
 'Bathtub',
 'Bed',
 'Billiards room',
 'Broadband Internet',
 'Club',
 'Clubhouse',
 'Colonial Building',
 'Corner Unit',
 'DVD Player',
 'Dishwasher',
 'Driving range',
 'Dryer',
 'Fridge',
 'Function room',
 'Game room',
 'Ground Floor',
 'Hairdryer',
 'High Floor',
 'ISDN',
 'Internet Connection',
 'Iron / Ironing Board',
 'Jacuzzi',
 'Karaoke',
 'Kitchen',
 'Launderette',
 'Library',
 'Lounge',
 'Low Floor',
 'Maidsroom',
 'Mini golf range',
 'Mini-Mart',
 'Multi-purpose hall',
 'Original Condition',
 'Outdoor Patio',
 'Own Chiller Plant',
 'Pantry',
 'Pavillion',
 'Penthouse',
 'Playground',
 'Private Garden',
 'Private Pool',
 'Putting Green',
 'Reflexology Path',
 'Renovated',
 'Roof Terrace',
 'SCV/SMATV System',
 'Sea View',
 'Sky Lounge',
 'Squash court',
 'Tennis cou

In [170]:
all_columns = model["preprocessor"].transformers[0][2] +list(model['preprocessor'].transformers_[1][1].categories_[0]) + list(model['preprocessor'].transformers_[1][1].categories_[1])

In [197]:
model['preprocessor'].transformers_[1][1].categories_

[array(['AP Thailand - เอพี (ไทยแลนด์) จำกัด (มหาชน)',
        'Ananda Development - อนันดา ดีเวลลอปเม้นท์ จํากัด (มหาชน)',
        'L.P.N. Development - แอล.พี.เอ็น.ดีเวลลอปเม้นท์ จำกัด (มหาชน)',
        'Land and Houses - แลนด์ แอนด์ เฮ้าส์ จำกัด (มหาชน)',
        'Magnolia Quality Development - แมกโนเลีย ควอลิตี้ ดีเวล็อปเม้นต์ คอร์ปอเรชั่น จำกัด',
        'Major Development - เมเจอร์ ดีเวลลอปเม้นท์ จำกัด (มหาชน)',
        'Noble Development - โนเบิล ดีเวลลอปเมนท์ จำกัด (มหาชน)',
        'Pace Development - เพซ ดีเวลลอปเมนท์ คอร์ปอเรชั่น จำกัด (มหาชน)',
        'Pruksa Real Estate - พฤกษา เรียลเอสเตท จำกัด (มหาชน)',
        'Quality Houses - ควอลิตี้ เฮ้าส์ จำกัด (มหาชน)',
        'Raimon Land - ไรมอน แลนด์ จำกัด (มหาชน)',
        'SC Asset - เอสซี แอสเสท คอร์ปอเรชั่น จำกัด (มหาชน)',
        'Sansiri - แสนสิริ จำกัด (มหาชน)',
        'Supalai - ศุภาลัย จำกัด (มหาชน)',
        'TCC Capital land - ทีซีซี แคปปิตอล แลนด์ จำกัด',
        'The Icon Siam Superlux Residence - ดิ ไอคอนสยาม ซ

In [227]:
load_transform(pipeline_name)

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', StandardScaler())]),
                                 ['bedroom', 'bathroom', 'sqm', 'built_year',
                                  'luxuary', 'public', 'BRT_distance',
                                  'BTS_distance', 'MRT_distance',
                                  'APL_distance', 'Air-Conditioning',
                                  'Audio System', 'BBQ pits',
                                  'Basketball court', 'Bathtub', 'Bed',
                                  'Billiards room', 'Broadband Internet',
                                  'Club', 'Clubhouse', 'Colonial Building',
                                  'Corner Unit', 'DVD Player', 'Dishwasher',
                                  'Driving range', 'Dryer', 'Fridge',
                 

In [177]:
def _explainer(task, target):
    feature_names = all_columns
    if task == "classification":
        class_names = y_train[target].unique()
        # Fit the Explainer on the training data set using the LimeTabularExplainer 
        explainer = LimeTabularExplainer(load_transform(pipeline_name).fit_transform(X_train), feature_names = feature_names, class_names = class_names, mode = task, categorical_features = check_type(X_train))
    else:
        explainer = LimeTabularExplainer(load_transform(pipeline_name).fit_transform(X_train), feature_names = feature_names, mode = task)
    return explainer

In [178]:
_explainer(task, target)

In [198]:
all_columns = []
for i in range(len(model["preprocessor"].transformers_)):
    if model["preprocessor"].transformers_[i][0] == "num":
        all_columns.extend(model["preprocessor"].transformers[i][2])
    else:
        for j in range(len(model['preprocessor'].transformers_[1][1].categories_)):
            all_columns.extend(list(model['preprocessor'].transformers_[1][1].categories_[j]))

In [199]:
all_columns

['bedroom',
 'bathroom',
 'sqm',
 'built_year',
 'luxuary',
 'public',
 'BRT_distance',
 'BTS_distance',
 'MRT_distance',
 'APL_distance',
 'Air-Conditioning',
 'Audio System',
 'BBQ pits',
 'Basketball court',
 'Bathtub',
 'Bed',
 'Billiards room',
 'Broadband Internet',
 'Club',
 'Clubhouse',
 'Colonial Building',
 'Corner Unit',
 'DVD Player',
 'Dishwasher',
 'Driving range',
 'Dryer',
 'Fridge',
 'Function room',
 'Game room',
 'Ground Floor',
 'Hairdryer',
 'High Floor',
 'ISDN',
 'Internet Connection',
 'Iron / Ironing Board',
 'Jacuzzi',
 'Karaoke',
 'Kitchen',
 'Launderette',
 'Library',
 'Lounge',
 'Low Floor',
 'Maidsroom',
 'Mini golf range',
 'Mini-Mart',
 'Multi-purpose hall',
 'Original Condition',
 'Outdoor Patio',
 'Own Chiller Plant',
 'Pantry',
 'Pavillion',
 'Penthouse',
 'Playground',
 'Private Garden',
 'Private Pool',
 'Putting Green',
 'Reflexology Path',
 'Renovated',
 'Roof Terrace',
 'SCV/SMATV System',
 'Sea View',
 'Sky Lounge',
 'Squash court',
 'Tennis cou

In [189]:
model["preprocessor"].transformers_

[('num',
  Pipeline(steps=[('imputer', SimpleImputer(strategy='median')),
                  ('scaler', StandardScaler())]),
  ['bedroom',
   'bathroom',
   'sqm',
   'built_year',
   'luxuary',
   'public',
   'BRT_distance',
   'BTS_distance',
   'MRT_distance',
   'APL_distance',
   'Air-Conditioning',
   'Audio System',
   'BBQ pits',
   'Basketball court',
   'Bathtub',
   'Bed',
   'Billiards room',
   'Broadband Internet',
   'Club',
   'Clubhouse',
   'Colonial Building',
   'Corner Unit',
   'DVD Player',
   'Dishwasher',
   'Driving range',
   'Dryer',
   'Fridge',
   'Function room',
   'Game room',
   'Ground Floor',
   'Hairdryer',
   'High Floor',
   'ISDN',
   'Internet Connection',
   'Iron / Ironing Board',
   'Jacuzzi',
   'Karaoke',
   'Kitchen',
   'Launderette',
   'Library',
   'Lounge',
   'Low Floor',
   'Maidsroom',
   'Mini golf range',
   'Mini-Mart',
   'Multi-purpose hall',
   'Original Condition',
   'Outdoor Patio',
   'Own Chiller Plant',
   'Pantry',
   